In [ ]:
#!pip install -Uqq fastbook
#!pip install voila

In [3]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

In [25]:
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner(path/'export.pkl')

In [6]:
img = PILImage.create('images/Bears/black/Image_1.jpg')

In [7]:
pred,pred_idx,probs = learn_inf.predict(img)

In [14]:
out_pl = widgets.Output()
out_pl.clear_output()

In [19]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [20]:
btn_run = widgets.Button(description='Classify')

In [21]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
btn_run.on_click(on_click_classify)

In [22]:
btn_upload = widgets.FileUpload()

In [23]:
VBox([widgets.Label('Select your lego!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [24]:
#!jupyter serverextension enable voila --sys-prefix

Enabling: voila
- Writing config: /opt/conda/envs/fastai/etc/jupyter
    - Validating...
      voila 0.2.4 OK
